# Parametric Regression  

In this notebook, we attemps to find optimal parameters for the model by using various nonlinear parametric methods

In [1]:
import os
import time
import shutil
import contextlib
import numpy as np
import matplotlib.pyplot as plt


from helper import *
from models import *
from solvers.euler import *
from data_generator import *
from solvers.gn_solver import *
from solvers.rk_solvers import *


print(f"Your machine has {os.cpu_count()} cores.")


Your machine has 8 cores.


## 1 Gauss Newton

First obtain some data as a list of tuples `total`. The first component is the initial conditions of `(201,2)` while the second is of the shape `(1000, 201, 2)`

In [3]:
N = 10

start = time.time()

total_model_4 = generateData(model4,
        num_data = 10,
        init_sty = 'random',
        times = (0, 20),
        params = {'no. of prey': N, 
    'kappa for prey': 0.5, 
    'attraction of prey a': 0, 
    'repulsion of prey b_1': 0.15, 
    'repulsion of pred b_2': 0.55,
    'p1 spotted': 0.8,
    'p2 not spotted':0.4,
    'angle_prey': np.cos(np.pi / 3),
    'angle_pred': np.cos(np.pi / 2.5), 
    'num_neighbours': 5,
    'attraction of pred c': 10, 
    'exponent of dist pred p': 3},
        steps = 1000,
        second_order = False,
        method = 'rk2',
        return_vel = False,
        cores = 1,
        flattened=True)
end = time.time()
print(f"Time taken: {end-start} seconds.")

Trying to use multiprocessing...
Multiprocessing successful.
Time taken: 2.807790994644165 seconds.


Alternatively, we generate data using `model1`, the simplest of all models.

In [4]:
# trying a simpler model
start = time.time()

total = generateData(model1,
        num_data = 10,
        init_sty = 'random',
        times = (0, 20),
        params = {'no. of prey': N, 
    'kappa for prey': 0.5, 
    'attraction of prey a': 1, 
    'repulsion of prey b_1': 1, 
    'repulsion of pred b_2': 0.07, 
    'attraction of pred c': 10, 
    'exponent of dist pred p': 1.2},
        steps = 1000,
        second_order = False,
        method = 'rk2',
        return_vel = False,
        cores = 1,
        flattened=True)
end = time.time()
print(f"Time taken: {end-start} seconds.")

Trying to use multiprocessing...
Multiprocessing successful.
Time taken: 1.972480058670044 seconds.


We still need a function that takes in parameter vector instead of a dictionary for regression; we take `attraction of prey a`, `repulsion of prey b_1`, `repulsion of pred b_2` as parameters for regression.

In [5]:
def fModel4(x, vec):
    """
    Returns a flattened output.  

    Args:  
        - x: ndarray, initial conditions
        - vec: ndarray, parameters of length 3
    
    """  
    params = {'no. of prey': N, 
    'kappa for prey': 0.5, 
    'attraction of prey a': vec[0], 
    'repulsion of prey b_1': vec[1], 
    'repulsion of pred b_2': vec[2],
    'p1 spotted': 0.8,
    'p2 not spotted':0.4,
    'angle_prey': np.cos(np.pi / 3),
    'angle_pred': np.cos(np.pi / 2.5), 
    'num_neighbours': 5,
    'attraction of pred c': 10, 
    'exponent of dist pred p': 3}

    N = 100
    times = (0,20)
    steps = 1000
    start, end = times  
    h = (end - start) / steps
    x = x.reshape(-1, 2) # reshape to coordinate form of initial conditions
    y = rk2(model4, x, h, 2*N+1, times, params, return_vel=False, verbose=False)

    return np.ravel(y)

We try to extract just one parameter, `attraction of prey a`

Feed in one sample and try the `GaussNewtonSolver`. Note it might be better to drop the zero values in the input and to remove some configurations at some time steps of $y$

In [6]:
def fModel1(x, vec):
    """
    Returns a flattened output.  

    Args:  
        - x: ndarray, initial conditions
        - vec: ndarray, parameters of length 1
    
    """  
    N = 10

    params = {'no. of prey': N, 
    'kappa for prey': 0.5, 
    'attraction of prey a': vec[0], 
    'repulsion of prey b_1': 1, 
    'repulsion of pred b_2': 0.07, 
    'attraction of pred c': 10, 
    'exponent of dist pred p': 1.2}
    
    times = (0,20)
    steps = 1000
    start, end = times  
    h = (end - start) / steps
    x = x.reshape(-1, 2) # reshape to coordinate form of initial conditions
    y = rk2(model1, x, h, 2*N+1, times, params, return_vel=False, verbose=False)

    return np.ravel(y)[::100]

We also apply the same augmentation to data

In [7]:
x0 = total[0][0]
y0 = total[0][1]  
print(y0.shape)
# only keep times that are 100 apart  
y0 = y0[::100]
print(len(y0))

(42000,)
420


In [8]:
def reduceTimesteps(xy_pair, timesteps=100):
    """
    Reduces the number of timesteps by a factor of timesteps.
    """
    return (xy_pair[0], xy_pair[1][::timesteps])

total = [reduceTimesteps(xy_pair) for xy_pair in total]

In [9]:
print(total[0][1].shape)

(420,)


Ideally the solver should return `attraction of prey a` as `1` with some acceptable error

In [10]:
guess = np.array([2])
gnsolver = GaussNewtonSolver(fModel1, max_solver_iter=10, step = 1e-3, max_line_search_iter=5, verbose=True)
gnsolver.fit(x0, y0, -5, guess, alpha = 0.1, beta=0.8)

  0%|          | 0/10 [00:00<?, ?it/s]

Starting trial 1


 10%|█         | 1/10 [00:04<00:44,  4.91s/it]

Max iteration reached for line search!
The step size is -1.3107200000000006
The descent direction is [0.02088798]
After trial 1, parameters are [1.97262171] 

Starting trial 2


 20%|██        | 2/10 [00:09<00:39,  4.89s/it]

...

 90%|█████████ | 9/10 [00:43<00:04,  4.86s/it]

Max iteration reached for line search!
The step size is -2.9230032746618137e-05
The descent direction is [0.05700314]
After trial 9, parameters are [1.95567137] 

Starting trial 10


100%|██████████| 10/10 [00:48<00:00,  4.86s/it]

Max iteration reached for line search!
The step size is -7.66247770432947e-06
The descent direction is [0.05700213]
After trial 10, parameters are [1.95567094] 



array([1.95567094])

To take this a step further, we perform the Gauss-Newton algorithm multiple times with more data

In [12]:
from regression.regressor import *  

x0 = [pair[0] for pair in total]
y0 = [pair[1] for pair in total]

t = -5 # initial descent step size

init_guess = np.array([2]) # initial guess for parameters

solver = "GaussNewton"  

solver_params = {
    "max_solver_iter": 10,
    "step": 1e-3,
    "max_line_search_iter": 5,
    "verbose": True
}

fit_params = {
    "alpha": 0.1,
    "beta": 0.8
}

params = regressor(x0, y0, t, fModel1, init_guess, solver, solver_params, fit_params)

Using GaussNewton solver


  0%|          | 0/10 [00:00<?, ?it/s]

Starting trial 1


Max iteration reached for line search!
The step size is -1.3107200000000006
The descent direction is [0.02088798]
After trial 1, parameters are [1.97262171] 

Starting trial 2


Max iteration reached for line search!
The step size is -0.3435973836800002
The descent direction is [0.02956899]
After trial 2, parameters are [1.96246188] 

Starting trial 3


...

Max iteration reached for line search!
The step size is -0.000425352958651174
The descent direction is [-6.9015032e-08]
After trial 7, parameters are [0.99999993] 

Starting trial 8


Max iteration reached for line search!
The step size is -0.00011150372599265338
The descent direction is [-6.89856847e-08]
After trial 8, parameters are [0.99999993] 

Starting trial 9


Max iteration reached for line search!
The step size is -2.9230032746618137e-05
The descent direction is [-6.8977991e-08]
After trial 9, parameters are [0.99999993] 

Starting trial 10


100%|██████████| 10/10 [08:07<00:00, 48.78s/it]

Max iteration reached for line search!
The step size is -7.66247770432947e-06
The descent direction is [-6.89759674e-08]
After trial 10, parameters are [0.99999993] 



In [13]:
params

array([0.99999993])